In [8]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with WRF data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime
import pandas as pd

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
#     m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
root_dir='/glade/u/home/hongli/scratch/2019_10_01gssha/ens_forc_wrf2'
wrf_raw_file = os.path.join(root_dir,'scripts/step1_asc_to_nc/WestWRF_daily_20171202_20180407.nc')
ens_dir = os.path.join(root_dir,'test_uniform_cv/046grids/outputs')
ens_mean_file = os.path.join(ens_dir,'ens_mean.nc')
ens_std_file = os.path.join(ens_dir,'ens_std.nc')

plot_date_start = '2018-02-18' #'2018-01-24' #'2018-02-18' #'2017-12-02'
plot_date_end = '2018-02-18' #'2018-01-24' #'2018-02-18' #'2018-04-07'
time_format = '%Y-%m-%d'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, '%Y-%m-%d')
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, '%Y-%m-%d')
plot_times = pd.date_range(start=plot_date_start_obj, end=plot_date_end_obj, freq='1D')

output_dir=os.path.join(root_dir,'scripts/step21_plot_spatial_ens')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
# ========================================================================================================================
# read historical wrf data
print('Read WRF data')
f_wrf = xr.open_dataset(wrf_raw_file)
wrf_time = pd.to_datetime(f_wrf['time'].values[:]).strftime(time_format)
wrf_lats = f_wrf['lat'] #(21)
wrf_lons = f_wrf['lon'] #(18)
wrf_prcp = f_wrf['prcp'].values[:] # (time, lat, lon). unit: mm/day
wrf_tmin = f_wrf['tmin'].values[:] # (time, lat, lon). unit: degC
wrf_tmax = f_wrf['tmax'].values[:] 

wrf_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in wrf_time])
wrf_tmean = 0.5*(wrf_tmax + wrf_tmin)
wrf_trange = wrf_tmax - wrf_tmin

# spatial mask
(nt_day,ny,nx)=np.shape(wrf_prcp)
mask = (np.ones((ny,nx))!=1)
mask[0:4,0]=True
mask[0,-1]=True
mask[-2,0:2]=True
mask[-1,0:10]=True
mask[-8:,-1]=True
mask = np.flipud(mask)

# ========================================================================================================================
# read ensemble data
print('Read ensemble mean/std')
f = xr.open_dataset(ens_mean_file)
pcp_ensmean = f['pcp'].values[:] # (time, lat, lon). unit: mm/day
tmean_ensmean = f['t_mean'].values[:] # unit: degC
trange_ensmean = f['t_range'].values[:] 

f = xr.open_dataset(ens_std_file)
pcp_ensstd = f['pcp'].values[:] # (time, lat, lon). unit: mm/day
tmean_ensstd = f['t_mean'].values[:] # unit: degC
trange_ensstd = f['t_range'].values[:] 

ens_time = pd.to_datetime(f['time'].values[:]).strftime(time_format)
ens_time = np.asarray([datetime.datetime.strptime(t, time_format) for t in ens_time])

#======================================================================================================
print('Plot')
for plot_time in plot_times:
    print(plot_time)
    wrf_time_mask = (wrf_time==plot_time)
    ens_time_mask= (ens_time==plot_time)

    # setup plot colorbar range for the plot_date
    vmin_prcp_mean=np.nanmin([np.nanmin(wrf_prcp[wrf_time_mask,:,:]), np.nanmin(pcp_ensmean[ens_time_mask,:,:])])
    vmax_prcp_mean=np.nanmax([np.nanmax(wrf_prcp[wrf_time_mask,:,:]), np.nanmax(pcp_ensmean[ens_time_mask,:,:])])
    
    vmin_t_mean_mean=np.nanmin([np.nanmin(wrf_tmean[wrf_time_mask,:,:]), np.nanmin(tmean_ensmean[ens_time_mask,:,:])])
    vmax_t_mean_mean=np.nanmax([np.nanmax(wrf_tmean[wrf_time_mask,:,:]), np.nanmax(tmean_ensmean[ens_time_mask,:,:])])
    
    vmin_t_range_mean=np.nanmin([np.nanmin(wrf_trange[wrf_time_mask,:,:]), np.nanmin(trange_ensmean[ens_time_mask,:,:])])
    vmax_t_range_mean=np.nanmax([np.nanmax(wrf_trange[wrf_time_mask,:,:]), np.nanmax(trange_ensmean[ens_time_mask,:,:])])

    # plot
    nrow = 3 # prcp, tmean, trange
    ncol = 3 # wrf, ens mean, ens std
    fig, ax = plt.subplots(nrow, ncol, figsize=(9,9*0.85))#, constrained_layout=True)

    llcrnrlon = wrf_lons[0,0]
    urcrnrlon = wrf_lons[-1,-1]
    llcrnrlat = wrf_lats[0,0]
    urcrnrlat = wrf_lats[-1,-1]
    lat_0=0.5*(llcrnrlat+urcrnrlat)
    lon_0=0.5*(llcrnrlon+urcrnrlon)
    (ny,nx)=np.shape(wrf_lons)

    for i in range(nrow):
        for j in range(ncol):

            # plot Basemap
            m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[i,j],lat_0,lon_0,ny,nx) # plot Basemap 

            # select data for each subplot
            # PCP (first row)
            if i == 0 and j == 0:
                data=wrf_prcp[wrf_time_mask,:,:]
                cmap=plt.cm.jet#Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(a) WRF Precip'
            elif i == 0 and j == 1:
                data=pcp_ensmean[ens_time_mask,:,:]
                cmap=plt.cm.jet#Blues
                vmin=vmin_prcp_mean
                vmax=vmax_prcp_mean
                title_str = '(b) Ens Precip Mean'
            elif i == 0 and j == 2:
                data=pcp_ensstd[ens_time_mask,:,:]
                cmap=plt.cm.jet#Blues
    #             vmin=vmin_prcp_std
    #             vmax=vmax_prcp_std
                title_str = '(c) Ens Precip Std'

            # T_MEAN (second row)
            elif i == 1 and j == 0:
                data=wrf_tmean[wrf_time_mask,:,:]
                cmap=plt.cm.jet#Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(d) WRF Tmean'
            elif i == 1 and j == 1:
                data=tmean_ensmean[ens_time_mask,:,:]
                cmap=plt.cm.jet#Reds
                vmin=vmin_t_mean_mean
                vmax=vmax_t_mean_mean
                title_str = '(e) Ens Tmean Mean'
            elif i == 1 and j == 2:
                data=tmean_ensstd[ens_time_mask,:,:]
                cmap=plt.cm.jet#Reds
    #             vmin=vmin_t_mean_std
    #             vmax=vmax_t_mean_std
                title_str = '(f) Ens Tmean Std'

            # T_RANGE (fifth row)
            elif i == 2 and j == 0:
                data=wrf_trange[wrf_time_mask,:,:]
                cmap=plt.cm.jet#Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(g) WRF Trange'
            elif i == 2 and j == 1:
                data=trange_ensmean[ens_time_mask,:,:]
                cmap=plt.cm.jet#Greens
                vmin=vmin_t_range_mean
                vmax=vmax_t_range_mean
                title_str = '(h) Ens Trange Mean'
            elif i == 2 and j == 2:
                data=trange_ensstd[ens_time_mask,:,:]
                cmap=plt.cm.jet#Greens #bwr
    #             vmin=vmin_t_range_std
    #             vmax=vmax_t_range_std
                title_str = '(i) Ens Trange Std'

            # plot data
            data[0,mask]=np.nan
            if j <=1:
                im1 = m.pcolormesh(wrf_lons,wrf_lats,data[0,:,:],shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)
            else:
                im1 = m.pcolormesh(wrf_lons,wrf_lats,data[0,:,:],shading='flat',latlon=True,cmap=cmap)

            # set title
            ax[i,j].set_title(title_str, fontsize='small', fontweight='semibold')

            # set colorbar
            cbar = m.colorbar(im1, ax=ax[i,j], location='right')
            if i == 0:
                cbar.set_label(label='(mm/day)', size='small', rotation='horizontal', labelpad=-20, y=1.1) #y=1.04
            elif i >= 1:
                cbar.set_label(label='($^\circ$C)', size='small', rotation='horizontal', labelpad=-20, y=1.1)
            cbar.ax.tick_params(labelsize='small') 

    # save plot
    fig.tight_layout(pad=0.1, h_pad=0.2, w_pad=0.70) # pad and h_pad configurations have no effects.
    output_filename = '046_gids_'+datetime.datetime.strftime(plot_time,time_format)+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=150)
#     plt.show()
    plt.close(fig)

print('Done')


Read WRF data
Read ensemble mean/std
Plot
2018-02-18 00:00:00
Done


In [6]:
data[0,mask]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [15]:
pcp_ensmean[ens_time_mask,:,:]

array([], shape=(0, 22, 19), dtype=float32)

In [17]:
ens_time,plot_time

(Index(['2017-12-02', '2017-12-03', '2017-12-04', '2017-12-05', '2017-12-06',
        '2017-12-07', '2017-12-08', '2017-12-09', '2017-12-10', '2017-12-11',
        ...
        '2018-03-29', '2018-03-30', '2018-03-31', '2018-04-01', '2018-04-02',
        '2018-04-03', '2018-04-04', '2018-04-05', '2018-04-06', '2018-04-07'],
       dtype='object', length=127),
 Timestamp('2018-01-24 00:00:00', freq='D'))